# Preamble

In [1]:
import numpy as np
import itertools

Some Matrix functions:

In [2]:
def dag(M):
    return np.conjugate(M.T)
def matInv(M):
    return np.linalg.inv(M)
def is_eq(A,B):
    return np.array_equal(A,B)

In [3]:
A = np.array([[0, 1.j],[-1.j, 0]])
print(A)
print(is_eq(dag(A), matInv(A)))

[[ 0.+0.j  0.+1.j]
 [-0.-1.j  0.+0.j]]
True


# Fusion Rules and F- and R- Symbol Calculation

## Fusion

Using the Verlinde formula to get S-matrix and then using that to extract the fusion rules $N_{\mathcal{A}\mathcal{B}}^{\mathcal{C}}$:

$\mathcal{S}_{\mathcal{A}\mathcal{B}} = \frac{1}{|G|}\sum_{h_i^A \in C_A, h_j^B \in C_B, h^A_i h^B_j = h^B_j h^A_i} \chi^A((x_i^A)^{-1}h_j^B x_i^A)\chi^B((x_j^B)^{-1}h_i^A x_j^B)$,

where $x_i^A h_1^A(x_i^A)^{-1} = h_1^A$ which is a $C_A$ representative [alt. $q_c r \bar{q}_c = c$].

$N_{\mathcal{A}\mathcal{B}}^{\mathcal{C}} = \sum_\mathcal{L} \frac{\mathcal{S}_{\mathcal{A}\mathcal{L}}\mathcal{S}_{\mathcal{B}\mathcal{L}}\mathcal{S}_{\mathcal{C}\mathcal{L}}^*}{\mathcal{S}_{\mathcal{0}\mathcal{L}}}$.

## F- and R-

Imagine we hace a space $\mathcal{H} = \mathcal{H}_b \otimes \mathcal{H}_a$ we can project down to the space of irreducible representations by $\mathcal{P}_{c}^{ba}(\ket{c, \mu}) \propto \sum_{h \in G}\sum_{z \in Z(r)}R^\dag_{\mu\mu}(z) B_h^b A_{q_c z \bar{q}_c}^b \otimes B_{h^{-1}c}^a A_{q_c z \bar{q}_c}^a$, where $\ket{c, \mu} \in \mathcal{H}_c$.

This allows us to define the splitting tree map: $\phi_c^{ba}: \mathcal{H}_c \rightarrow \mathcal{H}_b \otimes \mathcal{H}_a$ as $\phi_c^{ba}(\ket{c, \mu}) = \text{eig}(\mathcal{P}_{c}^{ba}(\ket{c, \mu}), \lambda = 1)$

The braiding action $\mathcal{B}_{ab}: \mathcal{H}_a\otimes\mathcal{H}_b\rightarrow\mathcal{H}_b\otimes\mathcal{H}_a$ is given by: $Flip \circ (\sum_g A_g^a \otimes B_g^b)$.

Combining the two facts we can evaluate the R-symbol as: $\mathcal{R}_c^{ab} \propto \phi_c^{ba}(\bra{c, \mu})\mathcal{B}_{ab}(\phi_c^{ab}(\ket{c, \mu}))$, with condition $|\mathcal{R}_c^{ab}| = 1$.

## $Q_8$

### Fusion Rule Calculations and Group Def

In [4]:
I = np.array([[1.j, 0],[0, -1.j]])
J = np.array([[0, -1],[1, 0]])
K = np.array([[0, -1.j],[-1.j, 0]])
G = np.zeros([2,2,8], complex)
G[:,:,0] = np.eye(2) # The Q_8 group
G[:,:,1] = -np.eye(2)
G[:,:,2] = I
G[:,:,3] = -I
G[:,:,4] = J
G[:,:,5] = -J
G[:,:,6] = K
G[:,:,7] = -K
g_lab = ['1','-1','i','-i','j','-j','k','-k']

C_e = np.array([0])  # Conj Classes
C_m = np.array([1])
C_1 = np.array([2,3])
C_2 = np.array([4,5])
C_3 = np.array([6,7])
# i - 1 ; j - 2 ; k - 3 !!!

Z_e = np.array([0, 1, 2, 3, 4, 5, 6, 7])  # Centers Classes
Z_m = np.array([0, 1, 2, 3, 4, 5, 6, 7])
Z_1 = np.array([0, 2, 1, 3])
Z_2 = np.array([0, 4, 1, 5])
Z_3 = np.array([0, 6, 1, 7])

X_e = np.array([0]) # Coset Representatives q_c
X_m = np.array([0])
X_1 = np.array([0,4])
X_2 = np.array([0,6])
X_3 = np.array([0,2])

In [5]:
# Def the Reps as Mats
def Triv(x):
    return np.array([[1]])
def A_1(x):
    if (x in C_2) or (x in C_3):
        return np.array([[-1]])
    else:
        return np.array([[1]])
def A_2(x):
    if (x in C_1) or (x in C_3):
        return np.array([[-1]])
    else:
        return np.array([[1]])
def A_3(x):
    if (x in C_1) or (x in C_2):
        return np.array([[-1]])
    else:
        return np.array([[1]])
def E(x):
    return G[:,:,x] # the 2d faithful rep
def Om_1(x):
    if x in [0]:
        return np.array([[1]])
    elif x in [1]:
        return np.array([[-1]])
    elif x in [2,4,6]:
        return np.array([[1.j]])
    elif x in [3,5,7]:
        return np.array([[-1.j]])
def Om_2(x):
    if x in [0]:
        return np.array([[1]])
    elif x in [1]:
        return np.array([[1]])
    elif x in [2,4,6]:
        return np.array([[-1]])
    elif x in [3,5,7]:
        return np.array([[-1]])
def Om_3(x):
    if x in [0]:
        return np.array([[1]])
    elif x in [1]:
        return np.array([[-1]])
    elif x in [2,4,6]:
        return np.array([[-1.j]])
    elif x in [3,5,7]:
        return np.array([[1.j]])

In [6]:
# Def chars of Reps
def triv(x):
    return 1
def a_1(x):
    if (x in C_2) or (x in C_3):
        return -1
    else:
        return 1
def a_2(x):
    if (x in C_1) or (x in C_3):
        return -1
    else:
        return 1
def a_3(x):
    if (x in C_1) or (x in C_2):
        return -1
    else:
        return 1
def e(x):
    if (x in C_e):
        return 2
    elif (x in C_m):
        return -2
    else:
        return 0
def om_1(x):
    if x in [0]:
        return 1
    elif x in [1]:
        return -1
    elif x in [2,4,6]:
        return 1.j
    elif x in [3,5,7]:
        return -1.j
def om_2(x):
    if x in [0]:
        return 1
    elif x in [1]:
        return 1
    elif x in [2,4,6]:
        return -1
    elif x in [3,5,7]:
        return -1
def om_3(x):
    if x in [0]:
        return 1
    elif x in [1]:
        return -1
    elif x in [2,4,6]:
        return -1.j
    elif x in [3,5,7]:
        return 1.j


In [7]:
# Def some AUX Info about the reps of D(Q8)
Z_s_Q = [Z_e, Z_m, Z_e, Z_e, Z_e, Z_m, Z_m, Z_m, Z_e, Z_m, Z_1, Z_1, Z_1, Z_1, Z_2, Z_2, Z_2, Z_2, Z_3, Z_3, Z_3, Z_3]
Dim_s_Q = [1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2]
C_mods_Q = [1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2]
chi_mods_Q = [1,1,1,1,1,1,1,1,2,2,1,1,1,1,1,1,1,1,1,1,1,1]
Z_reps_Q = [Triv, Triv, A_1, A_2, A_3, A_1, A_2, A_3, E, E, Triv, Om_1, Om_2, Om_3, Triv, Om_1, Om_2, Om_3, Triv, Om_1, Om_2, Om_3]

In [8]:
# Def Particles and some Fns
def q(c):
    Q_C = [0, 0, 0, 4, 0, 6, 0, 2] # This for q_c instead
    return Q_C[c]
def grp_i(i):
    return matInv(G[:,:,i])

def index(h):
    for i in range(8): # to be generalised
        if np.array_equal(h, G[:,:,i]):
            return i

def inv(x):
    for i in range(8): # to be generalised
        if np.array_equal(matInv(G[:,:,x]), G[:,:,i]):
            return i

def grp(x):
    return G[:,:, x] # USE G AS A GLOBAL VAR!!! ---- CAREFUL !

Particles = [(0, C_e, X_e, triv),(1, C_m, X_m, triv),(2, C_e, X_e, a_1),(3, C_e, X_e, a_2),(4, C_e, X_e, a_3),(5, C_m, X_m, a_1),(6, C_m, X_m, a_2),(7, C_m, X_m, a_3),(8, C_e, X_e, e),(9, C_m, X_m, e),\
    (10, C_1, X_1, triv), (11, C_1, X_1, om_1), (12, C_1, X_1, om_2), (13, C_1, X_1, om_3),\
        (14, C_2, X_2, triv), (15, C_2, X_2, om_1), (16, C_2, X_2, om_2), (17, C_2, X_2, om_3),\
            (18, C_3, X_3, triv), (19, C_3, X_3, om_1), (20, C_3, X_3, om_2), (21, C_3, X_3, om_3),]
Particles_Q = Particles
Particles_lab_Q = [('0', 'C_e', 'X_e', 'triv'),('1', 'C_m', 'X_m', 'triv'),('2', 'C_e', 'X_e', 'a_1'),('3', 'C_e', 'X_e', 'a_2'),('4', 'C_e', 'X_e', 'a_3'),('5', 'C_m', 'X_m', 'a_1'),('6', 'C_m', 'X_m', 'a_2'),('7', 'C_m', 'X_m', 'a_3'),('8', 'C_e', 'X_e', 'e'),('9', 'C_m', 'X_m', 'e'),\
    ('10', 'C_1', 'X_1', 'triv'), ('11', 'C_1', 'X_1', 'om_1'), ('12', 'C_1', 'X_1', 'om_2'), ('13', 'C_1', 'X_1', 'om_3'),\
        ('14', 'C_2', 'X_2', 'triv'), ('15', 'C_2', 'X_2', 'om_1'), ('16', 'C_2', 'X_2', 'om_2'), ('17', 'C_2', 'X_2', 'om_3'),\
            ('18', 'C_3', 'X_3', 'triv'), ('19', 'C_3', 'X_3', 'om_1'), ('20', 'C_3', 'X_3', 'om_2'), ('21', 'C_3', 'X_3', 'om_3'),]

S = np.zeros([22,22], complex)
for i, C_i, X_i, char_i in Particles:
    for j, C_j, X_j, char_j in Particles:
        s = 0 
        for i_class, elem_i_class in enumerate(C_i):
            for j_class, elem_j_class in enumerate(C_j):
                if is_eq(grp(elem_i_class)@grp(elem_j_class), grp(elem_j_class)@grp(elem_i_class)):
                    s+=char_i(index(grp_i(q(elem_i_class))@grp(elem_j_class)@grp(q(elem_i_class))))*char_j(index(grp_i(q(elem_j_class))@grp(elem_i_class)@grp(q(elem_j_class))))
        S[i,j] = s # maybe conj

S_Q = S/8
# print('\n'.join([''.join(['{:4}'.format(item) for item in row]) 
#      for row in S]))
N_Q = np.zeros([22,22,22])
for I in range(22):
    for J in range(22):
        for K in range(22):
            for L in range(22):
                N_Q[I,J,K] += np.real(S_Q[I,L]*S_Q[J,L]*np.conj(S_Q[K,L])/S_Q[0,L])

Query the S matrix:

In [9]:
an_1 = 10
an_2 = 15
print(Particles_lab_Q[an_1],',', Particles_lab_Q[an_2])
print('S_{aa}='+str(S_Q[an_1, an_1]))
print('S_{bb}='+str(S_Q[an_2, an_2]))
print('S_{ab}='+str(S_Q[an_1, an_2]))
print('S_{ba}='+str(S_Q[an_2, an_1]))
print('S_{0a}='+str(S_Q[0, an_2]))
print('S_{a0}='+str(S_Q[an_2, 0]))
print('S_{0a}='+str(S_Q[0, an_1]))
print('S_{a0}='+str(S_Q[an_1, 0]))

('10', 'C_1', 'X_1', 'triv') , ('15', 'C_2', 'X_2', 'om_1')
S_{aa}=(0.5+0j)
S_{bb}=(-0.5+0j)
S_{ab}=0j
S_{ba}=0j
S_{0a}=(0.25+0j)
S_{a0}=(0.25+0j)
S_{0a}=(0.25+0j)
S_{a0}=(0.25+0j)


Query the fusion rules:

In [10]:
i_1 = 14 # Prints fusion rules
i_2 = 14
print(Particles_lab_Q[i_1],'x', Particles_lab_Q[i_2], '=')
for i, indicator in enumerate(N_Q[i_1,i_2,:]):
    if indicator != 0:
        print(int(indicator),'x', Particles_lab_Q[i])
S_r = 8*np.real(S)

('14', 'C_2', 'X_2', 'triv') x ('14', 'C_2', 'X_2', 'triv') =
1 x ('0', 'C_e', 'X_e', 'triv')
1 x ('1', 'C_m', 'X_m', 'triv')
1 x ('3', 'C_e', 'X_e', 'a_2')
1 x ('6', 'C_m', 'X_m', 'a_2')


### Defining $D(Q_8)$ representation matrices



#### $A_g:$

In [11]:
def a_mat(an_type, g):
    # Basis is |c \mu> 
    d = Dim_s_Q[an_type]
    chi_mod = chi_mods_Q[an_type]
    C_mod = C_mods_Q[an_type]
    Z_set = Z_s_Q[an_type]
    Z_rep = Z_reps_Q[an_type]
    (id, C, X, char) = Particles_Q[an_type]
    # use q(c) instead of X set
    A = np.zeros([d, d], dtype=complex)
    for i_c, c in enumerate(C):
        for mu in range(chi_mod):
            for i_cp, c_prim in enumerate(C):
                for nu in range(chi_mod):
                    if is_eq(grp(c_prim), grp(g)@grp(c)@grp_i(g)):
                        rep_map = Z_rep(index( grp_i(q(c_prim))@grp(g)@grp(q(c)) ))
                        mat_elem = rep_map[mu,nu]
                        A[i_cp*chi_mod+nu, i_c*chi_mod+mu]=mat_elem
    return A


#### $B_h:$

In [12]:
def b_mat(an_type, g):
    # Basis is |c \mu> 
    d = Dim_s_Q[an_type]
    chi_mod = chi_mods_Q[an_type]
    C_mod = C_mods_Q[an_type]
    Z_set = Z_s_Q[an_type]
    Z_rep = Z_reps_Q[an_type]
    (id, C, X, char) = Particles_Q[an_type]
    # use q(c) instead of X set
    B = np.zeros([d, d], dtype=complex)
    for i_c, c in enumerate(C):
        for mu in range(chi_mod):        
            if g == c:
                B[i_c*chi_mod+mu, i_c*chi_mod+mu]=1
    return B

#### Query the Matrices:

In [13]:
an = 11
g = 1
print(Particles_lab_Q[an])
print(g_lab[g])
print(a_mat(an, g))
print(b_mat(an, g))

[[ 0.+1.j  0.+0.j]
 [ 0.+0.j -0.-1.j]] [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]


### The fusion projectors and plitting tree maps

In [14]:
def fusion_products(an_1, an_2):
    res = []
    for i, indicator in enumerate(N_Q[an_1,an_2,:]):
        if indicator != 0:
            res += [i]
    return res

def do_they_fuse(an_1, an_2, an_3):
    return an_3 in fusion_products(an_1, an_2)

def proj_norm(P):
    a = np.trace(P)
    b = np.trace(P@P)
    return P*a/b

def splitting_tree(an_1, an_2, an_3, id): #Only good if id = 0
    Pro = rep_proj_spec(an_1, an_2, an_3, id)
    w, v = np.linalg.eig(Pro)
    for i_lam, lam in enumerate(w):
        if lam == 1:
            Vec = v[:,i_lam]
    return Vec

def splitting_tree_proper(an_1, an_2, an_3): #Returns a properlt gauge fixed basis
    d_3 = Dim_s_Q[an_3]
    d_2 = Dim_s_Q[an_2]
    d_1 = Dim_s_Q[an_1]
    Basis_unfixed = []
    order = np.size(G[0,0,:])
    for i_3 in range(d_3):
        Basis_unfixed += [splitting_tree(an_1, an_2, an_3, i_3)]
    Basis_fixed = Basis_unfixed
    if d_3 == 1:
        return Basis_fixed
    else:
        for i_3 in range(1,d_3):
            not_done = 1
            for g in range(order):
                if not_done:
                    Vec_3 = np.zeros([d_1*d_2], dtype=complex)
                    for mat_mul in range(d_3):
                        Vec_3 += a_mat(an_3, g)[0,mat_mul]*Basis_unfixed[mat_mul]
                    Vec_12 = np.kron(a_mat(an_1, g), a_mat(an_2, g))@Basis_unfixed[0]
                    overlap_12 = np.vdot(Basis_unfixed[i_3], Vec_3)
                    overlap_3 = np.vdot(Basis_unfixed[i_3], Vec_12)
                    if overlap_12 != 0 and overlap_3 != 0:
                        Basis_fixed[i_3] = Basis_unfixed[i_3]*overlap_12/overlap_3
                        not_done = 0
    return Basis_fixed




def rep_proj(an_1, an_2, an_3):
    d_1 = Dim_s_Q[an_1]
    d_2 = Dim_s_Q[an_2]
    Z_3 = Z_s_Q[an_3]
    (id_3, C_3, X_3, char_3) = Particles_Q[an_3]
    Res = np.zeros([d_1*d_2,d_1*d_2], dtype=complex)
    for c in C_3:
        for z in Z_3:
            for h in range(8):
                Res += np.conjugate(char_3(z))*np.kron( b_mat(an_1, index( grp_i(h)@grp(c) ) ) @ a_mat(an_1,index( grp(q(c))@grp(z)@grp_i(q(c)) )),
                b_mat(an_2,h) @ a_mat(an_2,index( grp(q(c))@grp(z)@grp_i(q(c)) )) )
    return proj_norm(Res) # Normalisation issues :(

def rep_proj_spec(an_1, an_2, an_3, id):# Basis is |c \mu> # NOT SURE if the FORM is correct 
    d_1 = Dim_s_Q[an_1]
    d_2 = Dim_s_Q[an_2]
    Z_3 = Z_s_Q[an_3]
    chi_mod = chi_mods_Q[an_3]
    Z_rep = Z_reps_Q[an_3]
    (id_3, C_3, X_3, char_3) = Particles_Q[an_3]
    Res = np.zeros([d_1*d_2,d_1*d_2], dtype=complex)
    i_c = int(np.floor(id/chi_mod))
    c = C_3[i_c]
    mu = np.mod(id,chi_mod)
    for z in Z_3:
        for h in range(8):
            Res += Z_rep(z)[mu,mu].conj()*np.kron( b_mat(an_1, # Investigate does this one work???
            index( grp_i(h)@grp(c) ) ) @ a_mat(an_1,
            index( grp(q(c))@grp(z)@grp_i(q(c)) )),
            b_mat(an_2,h) @ a_mat(an_2,index( grp(q(c))@grp(z)@grp_i(q(c)) )) )
    return proj_norm(Res) # Normalisation issues :(



In [15]:
np.size(G[0,0,:])

8

### The R-sym Calculation:

In [16]:
def r_symbol(an_1, an_2, an_3):
    d_1 = Dim_s_Q[an_1]
    d_2 = Dim_s_Q[an_2]
    T = np.zeros([d_1*d_2, d_1*d_2], dtype=complex)
    # T_k = np.zeros([d_1*d_2, d_1*d_2], dtype=complex)
    for g in range(8):
        T += np.kron(a_mat(an_1,g), b_mat(an_2,g))
        # T_k += np.kron(b_mat(an_2,g), a_mat(an_1,g))
    # print('apply', T)
    F = np.zeros([d_1*d_2, d_1*d_2])
    for i in range(d_1):
        for j in range(d_2):
            for m in range(d_2):
                for n in range(d_1):
                    if i == n and j == m:
                        F[m*d_1+n, i*d_2 + j] = 1

    # Contrcut the Splitting Tree Basis!
    Vec_1 = splitting_tree(an_1, an_2, an_3,0)
    Vec_2 = splitting_tree(an_2, an_1, an_3,0) 
    Vec_2 = T@F@Vec_2 # The other way gives 0
    # print(Vec_1)
    # print(Vec_2)
    overlap = np.round(np.vdot(Vec_1, Vec_2),2) # issue !!!!
    # print(Vec_1, Vec_2)
    return overlap
    
def fusion_and_braid(an_1, an_2): # Depends on a lot of outside variables!!!
    # print(Particles_lab_Q[an_1],'x', Particles_lab_Q[an_2], '=')
    # Reduce text
    pr = []
    for i, indicator in enumerate(N_Q[an_1,an_2,:]):
        if indicator != 0:
            # Calculate the R-Symbol
            #print(int(indicator),'x', Particles_lab_Q[i], 'with R_symb', str(r_symbol(an_1,an_2, i)))
            pr += [(i, r_symbol(an_1,an_2, i))]
    print(an_1, 'x', an_2, '=', pr)
    return 0

The Results, quesries of fusion and R-symbols:

In [17]:
fusion_and_braid(21,21)

21 x 21 = [(0, -1j), (4, 1j), (5, -1j), (6, -1j)]


0

In [18]:
# for an_1, x, y, z in Particles_Q:
#     for an_2, xx, yy, zz in Particles_Q:
#         fusion_and_braid(an_1, an_2)

0 x 0 = [(0, (1+0j))]
0 x 1 = [(1, (1+0j))]
0 x 2 = [(2, (1+0j))]
0 x 3 = [(3, (1+0j))]
0 x 4 = [(4, (1+0j))]
0 x 5 = [(5, (1+0j))]
0 x 6 = [(6, (1+0j))]
0 x 7 = [(7, (1+0j))]
0 x 8 = [(8, (1+0j))]
0 x 9 = [(9, (1+0j))]
0 x 10 = [(10, (1+0j))]
0 x 11 = [(11, (1+0j))]
0 x 12 = [(12, (1+0j))]
0 x 13 = [(13, (1+0j))]
0 x 14 = [(14, (1+0j))]
0 x 15 = [(15, (1+0j))]
0 x 16 = [(16, (1+0j))]
0 x 17 = [(17, (1+0j))]
0 x 18 = [(18, (1+0j))]
0 x 19 = [(19, (1+0j))]
0 x 20 = [(20, (1+0j))]
0 x 21 = [(21, (1+0j))]
1 x 0 = [(1, (1+0j))]
1 x 1 = [(0, (1+0j))]
1 x 2 = [(5, (1+0j))]
1 x 3 = [(6, (1+0j))]
1 x 4 = [(7, (1+0j))]
1 x 5 = [(2, (1+0j))]
1 x 6 = [(3, (1+0j))]
1 x 7 = [(4, (1+0j))]
1 x 8 = [(9, (1+0j))]
1 x 9 = [(8, (1+0j))]
1 x 10 = [(10, (1+0j))]
1 x 11 = [(13, (1+0j))]
1 x 12 = [(12, (1+0j))]
1 x 13 = [(11, (1+0j))]
1 x 14 = [(14, (1+0j))]
1 x 15 = [(17, (1+0j))]
1 x 16 = [(16, (1+0j))]
1 x 17 = [(15, (1+0j))]
1 x 18 = [(18, (1+0j))]
1 x 19 = [(21, (1+0j))]
1 x 20 = [(20, (1+0j))]
1 x 21 =

### F- Symbols 


In [25]:
print(fusion_products(10,10))
print(do_they_fuse(10,10,4))
np.zeros([2])

[0, 1, 2, 5]
False


array([0., 0.])

In [20]:
def f_mat(an_a, an_b, an_c, an_d):
    d_a = Dim_s_Q[an_a]
    d_b = Dim_s_Q[an_b]
    d_c = Dim_s_Q[an_c]
    d_d = Dim_s_Q[an_d]
    x_ids = []
    y_ids = []
    print('x=')
    for x in fusion_products(an_b, an_c):
        if do_they_fuse(x,an_d,an_a):
            x_ids+=[x]
            print(Particles_lab_Q[x])
    print('y=')
    for y in fusion_products(an_c, an_d):
        if do_they_fuse(an_b,y,an_a):
            y_ids+=[y]
            print(Particles_lab_Q[y])
    F = np.zeros([len(x_ids), len(y_ids)], dtype=complex)

    for i_x, x in enumerate(x_ids):
        for i_y, y in enumerate(y_ids):
            d_x = Dim_s_Q[x]
            d_y = Dim_s_Q[y]
            i_vec_a = 0
            vec_xd = splitting_tree(x, an_d, an_a, i_vec_a)
            vec_by = splitting_tree(an_b, y, an_a, i_vec_a)
            vec_bcd_via_x = np.zeros([d_a*d_b*d_c], dtype=complex)
            vec_bcd_via_y = np.zeros([d_a*d_b*d_c], dtype=complex)
            x_basis = splitting_tree_proper(an_b, an_c, x)
            y_basis = splitting_tree_proper(an_c, an_d, y)
            for id_x in range(d_x):
                for id_d in range(d_d):
                    e_d = np.zeros([d_d])
                    e_d[id_d] = 1
                    vec_bcd_via_x += vec_xd[id_x*d_d + id_d]*np.kron(x_basis[id_x] ,e_d)
            for id_y in range(d_y):
                for id_b in range(d_b):
                    e_b = np.zeros([d_b])
                    e_b[id_b] = 1
                    vec_bcd_via_y += vec_by[id_b*d_y + id_y]*np.kron(e_b,y_basis[id_y])
            F[i_x, i_y] += np.vdot(vec_bcd_via_y, vec_bcd_via_x)
    F = np.round(F, 2)
    print('F=',F)
    return F

    

In [21]:
def f_mat_elem(an_a, an_b, an_c, an_d, x, y, i_vec_a): # use to debug
    d_a = Dim_s_Q[an_a]
    d_b = Dim_s_Q[an_b]
    d_c = Dim_s_Q[an_c]
    d_d = Dim_s_Q[an_d]
    F = 0.j
    d_x = Dim_s_Q[x]
    d_y = Dim_s_Q[y]
    vec_xd = splitting_tree(x, an_d, an_a, i_vec_a)
    vec_by = splitting_tree(an_b, y, an_a, i_vec_a)
    vec_bcd_via_x = np.zeros([d_a*d_b*d_c], dtype=complex)
    vec_bcd_via_y = np.zeros([d_a*d_b*d_c], dtype=complex)
    x_basis = splitting_tree_proper(an_b, an_c, x)
    y_basis = splitting_tree_proper(an_c, an_d, y)
    for id_x in range(d_x):
        for id_d in range(d_d):
            e_d = np.zeros([d_d])
            e_d[id_d] = 1
            vec_bcd_via_x += vec_xd[id_x*d_d + id_d]*np.kron(x_basis[id_x] ,e_d)
    for id_y in range(d_y):
        for id_b in range(d_b):
            e_b = np.zeros([d_b])
            e_b[id_b] = 1
            vec_bcd_via_y += vec_by[id_b*d_y + id_y]*np.kron(e_b, y_basis[id_y])
    print('x up', np.round(np.real(vec_bcd_via_x),2))
    print('y down',np.round(np.real(vec_bcd_via_y), 2))
    F += np.vdot(vec_bcd_via_x, vec_bcd_via_y)
    F = np.round(F, 2)
    return F

    

Query the F-matrices

In [22]:
F = f_mat(10, 14, 14, 10)

x=
('0', 'C_e', 'X_e', 'triv')
('1', 'C_m', 'X_m', 'triv')
y=
('18', 'C_3', 'X_3', 'triv')
('20', 'C_3', 'X_3', 'om_2')
F= [[ 0.71+0.j  0.71+0.j]
 [ 0.71+0.j -0.71+0.j]]


In [23]:
np.round(F@dag(F), 2)

array([[1.01+0.j, 0.  +0.j],
       [0.  +0.j, 1.01+0.j]])

In [24]:
f_mat_elem(14,10,14,10,20,20,0)

x up [-0.5  0.   0.   0.5  0.  -0.5  0.5  0. ]
y down [ 0.5  0.   0.  -0.5  0.   0.5 -0.5  0. ]


(-1+0j)